In [ ]:
from STalign import STalign
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd # for csv.
from matplotlib import cm
from matplotlib.lines import Line2D
import os
from os.path import exists,split,join,splitext
from os import makedirs
import glob
import requests
from collections import defaultdict
import nrrd
import torch
from torch.nn.functional import grid_sample
import tornado
import copy
import skimage
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pandas as pd
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.cm as cm
from scipy.ndimage import maximum_filter
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import os
from threadpoolctl import threadpool_limits
os.environ["MKL_NUM_THREADS"] = "50"
os.environ["NUMEXPR_NUM_THREADS"] = "50"
os.environ["OMP_NUM_THREADS"] = "50"
threadpool_limits(50)

In [ ]:
lipids = pd.read_csv('Data/STAlign_initialization_points.csv')
print(lipids.head())

In [ ]:
n_section = 2

In [ ]:
section = lipids.loc[lipids['Section'] == n_section,:]
image_array = np.zeros((320, 456))
for _, row in section.iterrows():
    z = row['z_index']
    y = row['y_index']
    intensity = row['density']
    image_array[y, z] = intensity
Allen_slice = maximum_filter(image_array, 2)
plt.imshow(Allen_slice)
ALLEN = Allen_slice
ALLEN = STalign.normalize(ALLEN)

In [ ]:
image_array = np.zeros((320, 456))
for _, row in section.iterrows():
    z = row['z_index']
    y = row['y_index']
    intensity = row['845.528']
    image_array[y, z] = intensity
lipid = maximum_filter(image_array, 2)
plt.imshow(lipid)
LIPID = lipid
LIPID = STalign.normalize(LIPID)

In [ ]:
zI = section['z_index']
yI = section['y_index']

In [ ]:
#Plot unrasterized/rasterized images
fig,ax = plt.subplots(1,2, figsize=(15,5))
ax[0].imshow(LIPID,  origin='lower')
ax[0].set_title('Lipids')

ax[1].imshow(ALLEN,  origin='lower')
ax[1].set_title('Allen')


In [ ]:
factor = 50
ZI = np.arange(0, 22800, factor)
YI = np.arange(0, 16000, factor)
xA = [YI,ZI]
xB = [YI,ZI]

In [ ]:
fig,ax = plt.subplots()
ax.hist(LIPID.ravel())
plt.xlabel('Intensity')
plt.ylabel('Number of Pixels')
plt.title('Intensity Histogram of Target Image')

In [ ]:
lipid_I = LIPID[None]
allen_J = ALLEN[None]


In [ ]:
from importlib import reload
reload(STalign)

In [ ]:
# initialize variables
scale_x = 0.23 #0.04 #default = 0.9
scale_y = 0.23 #0.045 #default = 0.9
scale_z = 1 #default = 0.9



scale_atlas = np.array([[scale_y,0],
                        [0,scale_x]])
L = np.array([[1.0,0.0],
             [0.0,1.0]])
L = np.matmul(L,scale_atlas)#np.identity(3)

In [ ]:
%%time
# run LDDMM
# specify device (default device for STalign.LDDMM is cpu)
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

# keep all other parameters default
params = {
            'niter': 2000,#4000,
            'device':'cpu',
            'epV': 1000,
            #'nt' : 10,
            'sigmaM' :0.5,
            #'L' : L,
            #'T' : T,
            #'sigmaR' : 100000000,
            #'epL' : 2e-7,
            #'epT' : 2,
            #'a' : 420
          }

out = STalign.LDDMM(xA,lipid_I,xB,allen_J,**params)

In [ ]:
A = out['A']
v = out['v']
xv = out['xv']
AI = out['AI']
Xs = out['Xs']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.cm as cm
Ishow_target = ALLEN
Ishow_target = Ishow_target.squeeze()
fig,ax = plt.subplots(2,2, figsize=(12,10))
ax0 = ax[0,0].imshow(Ishow_target, cmap = mpl.cm.Blues,alpha=0.9)
ax[0,0].set_title('Allen Slice 9')

ax1 = ax[0,1].imshow(AI[0])
#ax1 = ax[0,1].imshow(((AI-torch.amin(AI,(1,2))[...,None,None])/(torch.amax(AI,(1,2))-torch.amin(AI,(1,2)))[...,None,None]).permute(1,2,0)[:,:,0].clone().detach().cpu(), cmap = mpl.cm.Reds,alpha=0.9)
ax1 = ax[0,1].set_title('Our slice post STalign')

ax2 = ax[1,0].imshow(AI[0], cmap = mpl.cm.Reds,alpha=0.9)
#ax2 = ax[1,0].imshow(grid, cmap = mpl.cm.Greens,alpha=0.2)
ax[1,0].set_title('Our slice post STalign with Allen  contours')

ax3 = ax[1,1].imshow(AI[0], cmap = mpl.cm.Reds,alpha=0.9)
ax3 = ax[1,1].imshow(LIPID, cmap = mpl.cm.Greens,alpha=0.6)
ax[1,1].set_title('Overlayed before(green)-after(red) STalign')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from PIL import Image

def blend_images(alpha):
    fig,ax = plt.subplots(1,1, figsize=(15,5))
    ax.imshow(((AI-torch.amin(AI,(1,2))[...,None,None])/(torch.amax(AI,(1,2))-torch.amin(AI,(1,2)))[...,None,None]).permute(1,2,0)[:,:,0].clone().detach().cpu(), cmap = mpl.cm.Blues,alpha=alpha)
    ax.imshow(Ishow_target, cmap = mpl.cm.Reds,alpha=1-alpha)
    #ax.imshow(grid, cmap = mpl.cm.Greys,alpha=0.1)
    plt.axis('off')
    plt.show()

# Crea una barra di regolazione per l'intensità
interact(blend_images, alpha=widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Intensità'))

# Point-to-point alignment

In [ ]:
# ORIGINAL COORDINATES
zI = torch.tensor(zI.values)
yI = torch.tensor(yI.values)

device = 'cpu'
v = v.to(device) # VELOCITY FIELD
A = A.to(device)
zI = torch.tensor(zI, dtype=torch.double)
yI = torch.tensor(yI, dtype=torch.double)
zI_upscaled = zI*factor
yI_upscaled = yI*factor

# ALIGNED COORDINATES
tpointsI = STalign.transform_points_source_to_target(xv,v,A, np.stack([yI_upscaled, zI_upscaled], 1))

#switch tensor from cuda to cpu for plotting with numpy
if tpointsI.is_cuda:
    tpointsI = tpointsI.cpu()


section['y_index_new'] = tpointsI[:,0]/factor
section['z_index_new'] = tpointsI[:,1]/factor
print(section.head())

In [ ]:
section.to_csv(f'Output/section_{n_section}.csv', index=False)